In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
def extract_table(soup, table_id):
    table = soup.find('table', {'id': table_id})
    if not table:
        return None
    headers = [th.get_text(strip=True) for th in table.find_all('th')]
    rows = []
    for tr in table.find_all('tr')[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        if cells:
            rows.append(cells)
    return pd.DataFrame(rows, columns=headers)

def scrape_course_data(course_id):
  # TODO: Add CourseID to returned DF.

  # Make the HTTP request
  # response = requests.get(url)
  try:
    response = requests.get(
      'https://ncrdb.usga.org/courseTeeInfo',
      params={'CourseID': course_id},
      timeout=10
    )
  except Exception as e:
    print(f"❌ Error with course ID {course_id}: {e}")

  response.raise_for_status()  # Raise an error for bad status
  if response.status_code != 200:
      return None
  # Parse the HTML
  soup = BeautifulSoup(response.text, 'html.parser')

  # Extract the course-level metadata table (only one row)
  df_course_meta = extract_table(soup, 'gvCourseTees')

  # Check if the state is IL
  state = df_course_meta.iloc[0].get('State/Province', '')
  if state != 'IL':
    print(f"⚠️ Course ID: {course_id} not in IL")
    return None

  # Split club and course name into two cols then drop the old one
  df_course_meta[['Club', 'Course Name']] = df_course_meta['Club/Course Name'].str.split(' - ', n=1, expand=True)
  df_course_meta.drop(columns=['Club/Course Name'], inplace=True)

  # Extract the detailed tee-level table
  try:
    df_tee = extract_table(soup, 'gvTee')
    if df_tee is None or df_tee.empty:
        print("⭕ gvTee table not found or empty")
        return None
  except Exception as e:
    print(f"⭕ Error while extracting gvTee table: {e}")

  # Broadcast banana row to 5 rows
  df_course_meta_broadcast = pd.concat([df_course_meta]*len(df_tee), ignore_index=True)

  # Broadcast CourseID to rows
  df_course_id_broadcast =  pd.DataFrame([course_id] *len(df_tee), columns=["Course ID"])

  # Concatenate the new columns to apple
  df_tee_augmented = pd.concat([df_course_id_broadcast, df_course_meta_broadcast, df_tee], axis=1)

  time.sleep(0.25)  # Be polite to the server

  return df_tee_augmented


In [ ]:
def scrape_il_courses(min_id, max_id):
  all_data = pd.DataFrame()

  for course_id in range(min_id, max_id+1):  # First 1000 possible courses
    try:
      course_data = scrape_course_data(course_id)
      if course_data is None:
        continue
      all_data = pd.concat([all_data, course_data], ignore_index=True)
      print(f"✅ Scraped course ID {course_id}")

    except Exception as e:
      print(f"❌ Error with course ID {course_id}: {e}")
      continue

  return all_data


# Testing

## Small-Scale Testing

In [ ]:
# Small-scale testing (single course)
testing = scrape_course_data(7547)
# testing.head()

❌ gvTee table not found or empty


## Full-Scale Testing

In [ ]:
# BROKEN

# Finding min and max course_id
def find_min_max_course_id(starting_course_id, min=True):
  course_id_thresh = 0

  while course_id_thresh == 0:
    try:
      response = requests.get(
        'https://ncrdb.usga.org/courseTeeInfo',
        params={'CourseID': starting_course_id},
        timeout=10
      )
    except Exception as e:
      print(f"⚠️ Error with course ID {starting_course_id}: {e}")
    if response.status_code == 200:
      course_id_thresh = starting_course_id
    if min:
      starting_course_id += 1
    else:
      starting_course_id -= 1
    time.sleep(0.5)  # Be polite to the server
    print(course_id_thresh)

    return course_id_thresh

In [ ]:
course_id_min = find_min_max_course_id(0, min=True)
# course_id_max = find_min_max_course_id(0, min=False)

# course_id_min = 4
# course_id_max = 35765

# Hrs to pull every course in know DB space
# round((((course_id_max - course_id_min) / 2) / 60) / 60,2)

0


In [ ]:
# Full-scale testing (BE CAREFUL)
il_golf_course_db = scrape_il_courses(min_id=7665, max_id=10665)

#    Runs
# 1:    4 - 1004
# 2: 1005 - 2005

# Guessing some IL courses
# 3: 6664 - 7664
# 4: 7665 - 10665

# Earliest IL Course ID#: 7259

il_golf_course_db.shape

In [ ]:
master_db = il_golf_course_db

master_db.to_csv('illinois_courses.csv', index=False)

In [ ]:
db = pd.concat([master_db, il_golf_course_db], ignore_index=True)
db.shape

(2923, 23)

In [ ]:
db.to_csv('illinois_courses.csv', index=False)

In [ ]:
# il_golf_course_db.shape
il_golf_course_db.head()
# il_golf_course_db


NameError: name 'il_golf_course_db' is not defined

# Adding Geolocation Data

In [ ]:
# import csv
courses_df = pd.read_csv("illinois_courses.csv")
# cols_to_drop = ['temp_address', 'Unnamed: 21', 'CH', 'location', 'point']
# courses_df = courses_df.drop(columns=[col for col in cols_to_drop if col in courses_df.columns])

courses_df[0:100]
# course_testing_df = courses_df[0:1000]
# course_testing_df


,Course ID,City,State/Province,Club,Course Name,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,...,RatingF9,RatingB9,Front (9),Back (9),Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length
0,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black,M,72,73.7,99.5,...,36.9,36.8,36.9 / 139,36.8 / 138,49.9,49.6,139,139,615516,6961
1,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black/Gold,M,72,72.8,98.3,...,36.3,36.5,36.3 / 137,36.5 / 136,49.1,49.2,137,137,615513,6760
2,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold,M,72,71.0,95.9,...,35.4,35.6,35.4 / 134,35.6 / 132,47.9,48.0,134,134,177171,6365
3,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold/Silver,M,72,69.5,93.7,...,34.5,35.0,34.5 / 129,35.0 / 129,46.6,47.1,129,129,615531,6039
4,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Silver,M,72,67.6,90.4,...,33.6,34.0,33.6 / 123,34.0 / 124,44.9,45.5,123,123,177173,5625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7271,Woodstock,IL,Crystal Woods Golf Club,Crystal Woods Golf Club,Fun,F,72,65.0,90.7,...,32.4,32.6,32.4 / 110,32.6 / 110,45.3,45.4,110,110,725272,4278
96,7273,Algonquin,IL,Terrace Hill Golf Course,Terrace Hill Golf Course,Black,M,71,71.8,96.6,...,36.2,35.6,36.2 / 136,35.6 / 132,48.8,47.8,136,136,331571,6593
97,7273,Algonquin,IL,Terrace Hill Golf Course,Terrace Hill Golf Course,Blue,M,71,70.2,94.5,...,35.5,34.7,35.5 / 132,34.7 / 128,47.9,46.6,132,132,177248,6234
98,7273,Algonquin,IL,Terrace Hill Golf Course,Terrace Hill Golf Course,White,M,71,68.4,91.7,...,34.5,33.9,34.5 / 129,33.9 / 123,46.5,45.2,129,129,177249,5841


In [ ]:
# Function that uses geopy API to query club locations to extract addresses and coordinates
def get_business_geodata(courses_df, clean_name=False, use_club=True, use_city=True, use_state=True):
  # Running geopy
  geolocator = Nominatim(user_agent="business_lookup", timeout=10)

  # Adding rate limiter to be nice to the API
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

  # create tempory column for the geopy query string
  if clean_name and use_club:
    courses_df['temp_address'] = (
      courses_df['Club']
      .str.replace(' Golf Club', '')
      .str.replace('The', '')
    )
  elif use_club:
    courses_df['temp_address'] = courses_df['Club']
  else:
    pass

  if use_city:
    courses_df.loc[:,'temp_address'] += ", " + courses_df['City']

  if use_state:
    courses_df.loc[:,'temp_address'] += ", " + courses_df['State/Province']

  # courses_df['location'] = courses_df['temp_address'].progress_apply(geocode)
  courses_df.loc[courses_df['location'].isnull(), 'location'] = courses_df.loc[courses_df['location'].isnull(), 'temp_address'].progress_apply(geocode)

  # Extracting and formatting lat and long coords into [point] col
  courses_df['point'] = courses_df['location'].progress_apply(lambda p: str(p.point[0]) + "," + str(p.point[1]) if p else None)
  courses_df['lat'] = courses_df['location'].progress_apply(lambda p: p.point[0] if p else None)
  courses_df['lon'] = courses_df['location'].progress_apply(lambda p: p.point[1] if p else None)

  # Drop this column now that we no longer need it
  # courses_df = courses_df.drop('temp_address', axis=1)

  return courses_df


In [ ]:
# Grouping rows together by Club name to then run through get_business_geodata
# using club, city, and state
unique_clubs_df = courses_df[['Club', 'City', 'State/Province']].drop_duplicates()
unique_clubs_df

# Creating our geo columns
unique_clubs_df['location'] = None
unique_clubs_df['lat'] = None
unique_clubs_df['lon'] = None
unique_clubs_df['point'] = None

# Running get_business_geodata function with different params; this set of 3 seems to work pretty well
tqdm.pandas()
get_business_geodata(unique_clubs_df, clean_name=True)
get_business_geodata(unique_clubs_df, clean_name=True, use_city=False, use_state=False)
get_business_geodata(unique_clubs_df, clean_name=False, use_club = False) # Defaulting to just the city geodata instead of the specific club (only 19 rows at time of writing)
unique_clubs_df

100%|██████████| 326/326 [00:00<00:00, 297183.90it/s]


,Club,City,State/Province,location,lat,lon,point,temp_address
0,The Highlands of Elgin Golf Club,Elgin,IL,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
9,Lincolnshire Fields Country Club,Champaign,IL,"(Lincolnshire Fields Country Club, Meadow Broo...",40.087746,-88.299929,"40.0877458,-88.2999287","Lincolnshire Fields Country Club, Champaign, IL"
17,Fairways Golf Course,Rochelle,IL,"(Fairways Loch Ness Golf Club, Castle Heather ...",57.453642,-4.203532,"57.453642,-4.2035316","Fairways Golf Course, Rochelle, IL"
21,Butterfield Country Club,Oak Brook,IL,"(Butterfield Country Club, 2800, Midwest Road,...",41.836821,-87.976997,"41.8368208,-87.9769974","Butterfield Country Club, Oak Brook, IL"
57,Northmoor Country Club,Highland Park,IL,"(Northmoor Country Club, 820, Edgewood Road, R...",42.160263,-87.793893,"42.1602628,-87.7938931","Northmoor Country Club, Highland Park, IL"
...,...,...,...,...,...,...,...,...
2877,Lake Carroll Golf Course,Lake Carroll,IL,"(Carroll Lake Golf Course, Carroll Lake Road, ...",36.110124,-88.468822,"36.1101239,-88.4688218","Lake Carroll Golf Course, Lake Carroll, IL"
2884,Eagle Ridge Golf Course,Paris,IL,"(Eagle Ridge Golf Course, 928, Clinton Avenue,...",39.615479,-87.665026,"39.6154789,-87.6650257","Eagle Ridge Golf Course, Paris, IL"
2891,Briarwood Country Club,Deerfield,IL,"(Briarwood Country Club, 355, Deerfield Road, ...",42.161224,-87.837142,"42.1612243,-87.8371417","Briarwood Country Club, Deerfield, IL"
2904,Riverside Golf Club,North Riverside,IL,"(Riverside, Riverside County, California, Unit...",33.982495,-117.374238,"33.9824949,-117.374238","Riverside, North Riverside, IL"


Adding small increment to coords so each course at a club can be shown

In [ ]:
# Merge courses_df with our unique_clubs_df geodata
courses_df = pd.merge(courses_df, unique_clubs_df, on=['Club', 'City', 'State/Province'], how='left')
courses_df.head()

,Course ID,City,State/Province,Club,Course Name,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,...,Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,location,lat,lon,point,temp_address
0,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black,M,72,73.7,99.5,...,49.6,139,139,615516,6961,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
1,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black/Gold,M,72,72.8,98.3,...,49.2,137,137,615513,6760,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
2,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold,M,72,71.0,95.9,...,48.0,134,134,177171,6365,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
3,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold/Silver,M,72,69.5,93.7,...,47.1,129,129,615531,6039,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
4,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Silver,M,72,67.6,90.4,...,45.5,123,123,177173,5625,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"


In [ ]:
# Keep track of how many times each (lat, lon) has been seen
coord_counts = defaultdict(int)

def jitter_coords(row):
    lat, lon = row['lat'], row['lon']
    count = coord_counts[(lat, lon)]

    # Nudge if already seen
    if count > 0:
        # Add a small offset based on how many times it's been seen
        offset = 0.0001 * count  # adjust as needed
        lat += np.random.uniform(-offset, offset)
        lon += np.random.uniform(-offset, offset)

    coord_counts[(row['lat'], row['lon'])] += 1
    return pd.Series([lat, lon])

# Apply jitter
courses_df[['lat', 'lon']] = courses_df.apply(jitter_coords, axis=1)

In [ ]:
courses_df.head()

,Course ID,City,State/Province,Club,Course Name,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,...,Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,location,lat,lon,point,temp_address
0,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black,M,72,73.7,99.5,...,49.6,139,139,615516,6961,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
1,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black/Gold,M,72,72.8,98.3,...,49.2,137,137,615513,6760,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016954,-88.304482,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
2,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold,M,72,71.0,95.9,...,48.0,134,134,177171,6365,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016798,-88.304336,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
3,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold/Silver,M,72,69.5,93.7,...,47.1,129,129,615531,6039,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016909,-88.304400,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"
4,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Silver,M,72,67.6,90.4,...,45.5,123,123,177173,5625,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016866,-88.304573,"42.0169105,-88.3044313","Highlands of Elgin, Elgin, IL"


In [ ]:
# Cleaning up unused cols
cols_to_drop = ['temp_address', 'Unnamed: 21', 'CH']
courses_df = courses_df.drop(columns=[col for col in cols_to_drop if col in courses_df.columns])



In [ ]:
courses_df

,Course ID,City,State/Province,Club,Course Name,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,...,Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,location,lat,lon,point
0,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black,M,72,73.7,99.5,...,49.9,49.6,139,139,615516,6961,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016911,-88.304431,"42.0169105,-88.3044313"
1,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Black/Gold,M,72,72.8,98.3,...,49.1,49.2,137,137,615513,6760,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016954,-88.304482,"42.0169105,-88.3044313"
2,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold,M,72,71.0,95.9,...,47.9,48.0,134,134,177171,6365,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016798,-88.304336,"42.0169105,-88.3044313"
3,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Gold/Silver,M,72,69.5,93.7,...,46.6,47.1,129,129,615531,6039,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016909,-88.304400,"42.0169105,-88.3044313"
4,7259,Elgin,IL,The Highlands of Elgin Golf Club,The Highlands of Elgin Golf Club,Silver,M,72,67.6,90.4,...,44.9,45.5,123,123,177173,5625,"(Highlands of Elgin, 875, Sports Way, Elgin, E...",42.016866,-88.304573,"42.0169105,-88.3044313"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,8034,Westchester,IL,Chicago Highlands,Chicago Highlands,Green,M,72,68.5,91.8,...,45.4,46.4,121,121,529246,5646,"(Chicago Highlands Club, 2, West Cermak Road, ...",41.840248,-87.913833,"41.8405877,-87.9138809"
2919,8034,Westchester,IL,Chicago Highlands,Chicago Highlands,Red,M,72,65.8,88.1,...,43.4,44.7,115,115,680303,5049,"(Chicago Highlands Club, 2, West Cermak Road, ...",41.840049,-87.914590,"41.8405877,-87.9138809"
2920,8034,Westchester,IL,Chicago Highlands,Chicago Highlands,White,F,72,76.9,108.3,...,53.6,54.7,133,133,498034,6146,"(Chicago Highlands Club, 2, West Cermak Road, ...",41.840232,-87.913295,"41.8405877,-87.9138809"
2921,8034,Westchester,IL,Chicago Highlands,Chicago Highlands,Green,F,72,74.1,104.3,...,51.7,52.6,127,127,265807,5646,"(Chicago Highlands Club, 2, West Cermak Road, ...",41.841089,-87.914873,"41.8405877,-87.9138809"


# Final Export to CSV

In [ ]:
# Exporting our final dataframe to a CSV
courses_df.to_csv('illinois_courses_1.csv', index=False)

Adding small increment to coords so each course at a club can be shown

In [ ]:
type(unique_clubs_df.loc[:,'point'])

pandas.core.series.Series

### TOOLS

In [ ]:
# [TOOL] Searching for a specific course
courses_df[courses_df['Club'].str.contains('Crystal Woods Golf Club', case=False, na=False)]


# TO DO List
*   ~~Add in sleep time for pinging site~~
*   ~~Fix double error exception message~~
*   ~~Add filter for only IL courses~~
*   Add geo data (location, coords) using geopy
*   Fix some clubs not showing up on LS dash map
*   Fix get_business_geodata function to pull unique locations, then add those to the df so it isn't querying the same location multiple times
*   Might be missing other things, not 100% sure